In [1]:
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd
import signac
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter
hoomd.context.initialize("")

HOOMD-blue 2.9.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/18/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Lui, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


In [2]:
ptb7_path= "/home/emilyepstein/projects/ptb7-itic/ptb7-project/"
ptb7_project = signac.get_project(ptb7_path)
#specify which project (itic or ptb7)
project = ptb7_project

In [3]:
state_dict= {"size": "small",
             "density": 0.8,
             "kT_reduced": 1.0,
             "molecule": "PTB7_5mer_smiles",
             "process": "quench"}
job_list= project.find_jobs(state_dict)
for job in job_list:
    job_path = job.fn('trajectory.gsd')

In [4]:
def atom_type_pos(snap, atom_type):
    if not isinstance(atom_type, list):
        atom_type = [atom_type]
    positions = []
    for atom in atom_type:
        indices = np.where(snap.particles.typeid == snap.particles.types.index(atom))
        positions.append(snap.particles.position[indices])
    return np.concatenate(positions)

In [9]:
def dp_from_gsd(job_path, start=-30, stop=-1, atom_type='f'):
    f = gsd.pygsd.GSDFile(open(job_path, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    positions = []
    frame = len(trajectory)
    snap = hoomd.data.gsd_snapshot(job_path, frame-1)
    for snap in trajectory[start:stop]:
        if atom_type == 'all':
            points = snap.particles.position[:]
        else:
            points = atom_type_pos(snap, atom_type)
        positions.append(points)
    dp = freud.diffraction.DiffractionPattern(grid_size=1024,
                                  output_size=1024)
    dp.compute((box, points), view_orientation=np.array([1,0,0,0]))
    return dp, snap



In [10]:
points = snap.particles.position
box = freud.Box.from_box(snap.box)
dp = dp_from_gsd(job.fn('trajectory.gsd'), atom_type='f')
dp.compute((box, points), view_orientation=np.array([1,0,0,0]))

NameError: name 'snap' is not defined